# Temporal Linear Network (TLN) based VWAP execution algorithm

Python script for training and evaluating a TLN based model to optimize VWAP execution on META 1 minute stock data with market impact.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import keras
from tln import TLN
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
import tensorflow as tf
warnings.filterwarnings('ignore')

# Suppress TensorFlow warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

LOOKBACK: the number of minutes the model looks at to make predictions

HORIZON: number of future minutes for which the model predicts the allocation schedule

In [ ]:
# Model parameters
LOOKBACK = 120
HORIZON = 12
TOTAL_SHARES = 1_000_000

### Data Preprocessing

In [ ]:
# Load and preprocess data
df = pd.read_csv('../datasets/META_1min_firstratedata.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['avg_price'] = (df['high'] + df['low']) / 2
df.set_index('timestamp', inplace=True)
df = df.between_time('09:30', '16:00').copy()

### Feature Extraction

These are the features to be used to train the model. This is based on the paper's recommended insights and uses only past data, ensuring no forward looking.

In [ ]:
# Enhanced feature engineering (no forward-looking)
df['returns'] = df['avg_price'].pct_change().fillna(0)
df['log_returns'] = np.log(df['avg_price'] / df['avg_price'].shift(1)).fillna(0)
df['vol_5'] = df['returns'].rolling(5).std().fillna(0)
df['vol_30'] = df['returns'].rolling(30).std().fillna(0)
df['vol_scaled'] = df['volume'] / df['volume'].rolling(390*14, min_periods=390).mean().fillna(1)
df['hour'] = df.index.hour
df['minute'] = df.index.minute
df['dow'] = df.index.dayofweek
mins_from_open = (df.index.hour - 9) * 60 + (df.index.minute - 30)
df['intraday_pos'] = np.clip(mins_from_open / (6.5*60), 0, 1)
df['u_shape'] = 1 / (1 + 4 * (df['intraday_pos'] - 0.5)**2)

FEATURES = ['avg_price', 'volume', 'vol_scaled', 'log_returns', 'vol_5', 'vol_30',
            'hour', 'minute', 'dow', 'intraday_pos', 'u_shape']
NUM_FEATURES = len(FEATURES)

### Data Splitting into training (first 75%) and testing (last 25%)

In [ ]:
# Split data chronologically
dates = df.index.normalize().unique()
split = int(len(dates) * 0.75)
train_dates = dates[:split]
test_dates = dates[split:]

train_df = df[df.index.normalize().isin(train_dates)].copy()
test_df = df[df.index.normalize().isin(test_dates)].copy()

### Sequence Creation

Defines a function to generate input sequences (X) and targets (Y) for the model. It groups data by date, checks for sufficient length, and creates sliding windows of features and price/volume targets. This structure ensures sequences are contained within single trading days, avoiding cross-day bias and preparing data for time-series modeling.

In [ ]:
# Sequence creation (daily grouping, no leak)
def make_sequences(data, lookback, horizon):
    X, Y = [], []
    for _, day in data.groupby(data.index.date):
        if len(day) < lookback + horizon: continue
        feat = day[FEATURES].values
        tgt = day[['avg_price', 'volume']].values
        for i in range(len(day) - lookback - horizon + 1):
            X.append(feat[i:i+lookback])
            Y.append(tgt[i+lookback:i+lookback+horizon])
    return np.array(X), np.array(Y)

In [ ]:
X_train, y_train = make_sequences(train_df, LOOKBACK, HORIZON)
X_test, y_test = make_sequences(test_df, LOOKBACK, HORIZON)

### Scale features

This fits StandardScalers on the training data for each feature and transforms both train and test sets. Scaling normalizes the features to have mean 0 and standard deviation 1, which helps the model learn more effectively. Importantly, scalers are fit only on training data to prevent information leakage from the test set.

In [ ]:
# Scale features (fit on train only)
scalers = {f: StandardScaler().fit(X_train[:,:,j]) for j, f in enumerate(FEATURES)}
for j, f in enumerate(FEATURES):
    X_train[:,:,j] = scalers[f].transform(X_train[:,:,j])
    X_test[:,:,j] = scalers[f].transform(X_test[:,:,j])

### Custom Loss Function

The TLN architecture processes temporal sequences of market data to generate optimal allocation schedules, using a custom loss function that directly minimizes the quadratic difference between model VWAP and benchmark VWAP. This is the core feature of this model.

Building upon the original VWAP custom loss function, I implemented an enhanced version incorporating smoothness regularization:

* Loss = α × VWAP_slippage² + β × smoothness_penalty


In [ ]:
# Enhanced loss
def vwap_smooth_loss(y_true, y_pred):
    price = y_true[:,:,0]
    vol = y_true[:,:,1]

    # Use TensorFlow operations instead of NumPy
    m_val = tf.reduce_sum(y_pred * price, axis=1)
    m_sh = tf.reduce_sum(y_pred, axis=1)
    m_vwap = m_val / (m_sh + 1e-8)
    
    b_vwap = tf.reduce_sum(vol * price, axis=1) / (tf.reduce_sum(vol, axis=1) + 1e-8)
    
    sl = tf.square(m_vwap - b_vwap)
    smooth = tf.reduce_mean(tf.square(y_pred[:,1:] - y_pred[:,:-1]), axis=1)
    
    return sl + 0.05 * smooth

### Model Creation

Builds the TLN model architecture, starting with an input layer, followed by the TLN layer, dropout for regularization, flattening, a dense layer with L2 regularization, batch normalization, and a softmax output. This setup creates a network that processes sequential data to output allocation probabilities, with built-in mechanisms to prevent overfitting.

In [ ]:
# Model creation
inputs = keras.Input(shape=(LOOKBACK, NUM_FEATURES))
x = TLN(output_len=HORIZON, output_features=1, hidden_layers=3, use_convolution=True)(inputs)
x = keras.layers.Dropout(0.1)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(24, activation='relu', kernel_regularizer=keras.regularizers.l2(1e-3))(x)
x = keras.layers.BatchNormalization()(x)
outputs = keras.layers.Dense(HORIZON, activation='softmax')(x)
model = keras.Model(inputs, outputs)

model.compile(optimizer=keras.optimizers.Adam(clipnorm=1.0), loss=vwap_smooth_loss)

### Training with callbacks

Sets up training callbacks for early stopping, learning rate reduction, and scheduling, then fits the model on training data with validation split. The callbacks monitor validation loss to optimize training duration and learning rate dynamically. This improves model convergence and generalization without manual intervention.

In [ ]:
callbacks = [
    keras.callbacks.EarlyStopping(patience=15, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5),
    keras.callbacks.LearningRateScheduler(lambda e: 0.001 * 0.95**e)
]

history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2, callbacks=callbacks, verbose=1)

In [ ]:
def calculate_benchmark_vwap(daily_df):
    if daily_df.empty or daily_df['volume'].sum() == 0:
        return np.nan
    return (daily_df['avg_price'] * daily_df['volume']).sum() / daily_df['volume'].sum()

### Evaluation Function

Defines the backtesting function, grouping data by date and simulating trades using historical data only for volatility and volume calculations to avoid bias. It predicts schedules, applies market impact, accumulates trade values, and computes slippage metrics. The function returns a DataFrame of daily results for analysis.

In [ ]:
def evaluate_tln_strategy(test_df, model, total_shares_to_trade=1_000_000):
    daily_groups = test_df.groupby(test_df.index.date)
    results = []

    for date, day_data in daily_groups:
        if len(day_data) < LOOKBACK + HORIZON:
            continue

        benchmark_vwap = calculate_benchmark_vwap(day_data)
        if pd.isna(benchmark_vwap):
            continue

        total_value_of_trades = 0
        total_shares_traded = 0

        for i in range(len(day_data) - LOOKBACK):
            historical_data = day_data.iloc[:i + LOOKBACK + 1]
            daily_volatility = historical_data['avg_price'].std() if len(historical_data) > 1 else 0.0
            avg_daily_volume = historical_data['volume'].mean() if len(historical_data) > 0 else 1.0

            input_sequence = historical_data.iloc[-LOOKBACK:][FEATURES].values
            input_sequence = np.expand_dims(input_sequence, axis=0)

            predicted_schedule = model.predict(input_sequence, verbose=0)[0]

            shares_to_trade = predicted_schedule[0] * total_shares_to_trade
            base_price = day_data.iloc[i + LOOKBACK]['avg_price']

            impact = daily_volatility * np.sqrt(shares_to_trade / avg_daily_volume) if avg_daily_volume > 0 else 0.0
            execution_price = base_price + impact

            total_value_of_trades += shares_to_trade * execution_price
            total_shares_traded += shares_to_trade

        if total_shares_traded == 0:
            continue

        model_vwap = total_value_of_trades / total_shares_traded
        slippage = model_vwap - benchmark_vwap
        slippage_bps = (slippage / benchmark_vwap) * 10000 if benchmark_vwap != 0 else np.nan

        results.append({
            'date': date.strftime('%Y-%m-%d'),
            'benchmark_vwap': benchmark_vwap,
            'model_vwap': model_vwap,
            'slippage': slippage,
            'slippage_bps': slippage_bps
        })

    return pd.DataFrame(results)


In [ ]:
# Comprehensive metrics
def calculate_metrics(results_df):
    metrics = {}
    slippage_bps = results_df['slippage_bps'].dropna()
    if len(slippage_bps) == 0:
        return metrics

    metrics['avg_slippage_bps'] = slippage_bps.mean()
    metrics['std_slippage_bps'] = slippage_bps.std()
    metrics['win_rate'] = (slippage_bps < 0).mean() * 100
    metrics['sharpe_ratio'] = -metrics['avg_slippage_bps'] / metrics['std_slippage_bps'] if metrics['std_slippage_bps'] != 0 else 0
    metrics['max_drawdown'] = (slippage_bps.cumsum().cummax() - slippage_bps.cumsum()).max()
    metrics['mae'] = mean_absolute_error(results_df['benchmark_vwap'], results_df['model_vwap'])
    metrics['rmse'] = np.sqrt(mean_squared_error(results_df['benchmark_vwap'], results_df['model_vwap']))
    return metrics

In [ ]:
# Run evaluation
results_df = evaluate_tln_strategy(test_df, model)
metrics = calculate_metrics(results_df)

### Slippage Distribution

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(results_df['slippage_bps'].dropna(), bins=30, edgecolor='black')
plt.title('Slippage Distribution (bps)')
plt.xlabel('Slippage (bps)')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)
plt.show()